# Evaluate the box below to initialize the web UI.

In [ ]:
import json
import os
import pandas as pd
import pandas.io.sql as psql
import qgrid 
import ray
import redis
import sys

from bokeh.plotting import figure, show
from bokeh.layouts import layout, widgetbox, row, gridplot
from bokeh.models import ColumnDataSource, HoverTool, Div, CustomJS, Range1d
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import curdoc, output_notebook, push_notebook
from pandas.io.json import json_normalize

In [ ]:
redis_address = os.environ["REDIS_ADDRESS"]
ray.init(redis_address=redis_address)

# Task and Actor Information 

### Remote Function Information 


In [ ]:
fn_table = ray.global_state.function_table()
fn_list = []
if (len(fn_table) != 0): 
    for fn_id in fn_table:
        val = fn_table[fn_id]
        val["function_id"] = fn_id
        fn_list.append(val)
else: 
    fn_list.append({"DriverID": "None", "Module": "None", "Function": "None", "FunctionID": "None"})
qgrid.nbinstall(overwrite=True)
frame = pd.DataFrame(fn_list) 
frame.columns = ["DriverID", "Module", "Function", "FunctionID"]
qgrid.show_grid(frame)

### Task Information 

In [ ]:
tt = ray.global_state.task_table()
tt_list = list(tt.values())
for d in tt_list:
    d["TaskSpec"]["ReturnObjectIDs"] = [oid.hex() for oid in d["TaskSpec"]["ReturnObjectIDs"]]
    d["TaskSpec"]["Args"] = [arg.hex() if isinstance(arg, ray.local_scheduler.ObjectID) else arg for arg in d["TaskSpec"]["Args"]]

task_df = json_normalize(tt_list)
task_df.columns = ["Local Scheduler ID", "State", "Actor Counter", "ActorID", "Arguments", "DriverID", "FunctionID", 
                  "Parent Counter", "Parent Task ID", "Required CPUs", "Required GPUs", "Return Object IDs", "TaskID" ]
qgrid.show_grid(task_df)

### Task Profiles 

In [ ]:
task_info = ray.global_state.task_profiles()
task_breakdowns = dict()
if len(task_info) != 0: 
    for task_id, data in task_info.items(): 
        task_breakdowns[task_id] = dict()
        if "import_remote_start" and "import_remote_end" in data: 
            task_breakdowns[task_id]["import_remote_time"]  = data["import_remote_end"] - data["import_remote_start"] 
        if "store_outputs_end" and "store_outputs_start" in data: 
            task_breakdowns[task_id]["store_outputs"] = data["store_outputs_end"] - data["store_outputs_start"]
        task_breakdowns[task_id]["get_task_time"] = data["get_task_end"] - data["get_task_start"]
        task_breakdowns[task_id]["get_arguments"] = data["get_arguments_end"] - data["get_arguments_start"]
        task_breakdowns[task_id]["execute_time"] = data["execute_end"] - data["execute_start"]
        task_breakdowns[task_id]["acquire_lock"] = data["acquire_lock_end"] - data["acquire_lock_start"]
        task_breakdowns[task_id]["worker_id"] = data["worker_id"]
else: 
    task_breakdowns["None"] = {"Acquire Lock":"None", 
                               "Execute Time":"None", 
                               "Get Arguments":"None", 
                               "Get Task":"None", 
                               "Import Remote Function":"None", 
                               "Store Outputs":"None", 
                               "Worker ID":"None"}
df = pd.DataFrame.from_dict(task_breakdowns)
df_t = df.T
df_t.index.name = "Task ID"
df_t.columns = ["Acquire Lock", "Execute Time", "Get Arguments", "Get Task", "Import Remote Function", "Store Outputs", "Worker ID"] 
qgrid.show_grid(df_t)

### Task-Worker Placement Information 

In [ ]:
task_worker = dict()
for task_id, data in task_info.items(): 
    task_worker[task_id] = dict()
    task_worker[task_id]["worker_id"] = data["worker_id"]
    task_worker[task_id]["function_name"] = data["function_name"]
if len(task_worker) == 0: 
    task_worker["None"] = {"Function ID": "None", "Worker ID": "None"}
df = pd.DataFrame.from_dict(task_worker)
df_t = df.T 
df_t.index.name = "Task ID"
df_t.columns = ["Function Name", "Worker ID"]
qgrid.show_grid(df_t)

### Task Profile Visualization

In [ ]:
task_profs = task_info 
print(task_profs)
workers = dict()

counter = 0 
for task_id, data in task_profs.items(): 
    worker_id = data["worker_id"]
    if worker_id not in workers: 
        workers[worker_id] = counter
        counter += 1
    data["wid"] = workers[worker_id]        

all_times = []
for data in task_info.values(): 
    all_times.append(data["acquire_lock_start"])
    all_times.append(data["acquire_lock_end"])
    all_times.append(data["get_arguments_start"])
    all_times.append(data["get_arguments_end"])
    all_times.append(data["execute_start"])
    all_times.append(data["execute_end"])
    all_times.append(data["store_outputs_start"])
    all_times.append(data["store_outputs_end"])

min_time = min(all_times)
for data in task_info.values(): 
    data["acquire_lock_start"] = data["acquire_lock_start"] - min_time 
    data["acquire_lock_end"] = data["acquire_lock_end"] - min_time
    data["execute_start"] = data["execute_start"] - min_time  
    data["execute_end"] = data["execute_end"] - min_time  
    data["get_arguments_start"] = data["get_arguments_start"] - min_time  
    data["get_arguments_end"] = data["get_arguments_end"] - min_time  
    data["store_outputs_start"] = data["store_outputs_start"] - min_time  
    data["store_outputs_end"] = data["store_outputs_end"] - min_time  

object_dict = {oid.hex(): v for oid, v in ray.global_state.object_table().items()}
for oid, data in object_dict.items(): 
    tid = data["TaskID"]
    if tid in task_info: 
        task_info[tid]["object_id"] = oid 
        
output_notebook()
source = ColumnDataSource(data=dict(
                                    x=[],
                                    y=[],
                                    worker_id=[],
                                    task_id=[],
                                    function_name=[],
                                    get_arguments_start=[],
                                    get_arguments_end=[],
                                    acquire_lock_start=[],
                                    acquire_lock_end=[],
                                    execute_start=[],
                                    execute_end=[],
                                    store_outputs_start=[],
                                    store_outputs_end=[],
                                    wid=[],
                                    oid=[]
                                    ))
axis_map = {
    "worker_id": "worker_id",
    "time": "time",
}
 
hover = HoverTool(tooltips=[
    ("TaskID", "@task_id"),
    ("Function Name", "@function_name"),
    ("WorkerID", "@wid"),
    ("ObjectID", "@oid"),
    ("Variables", "@vars")
])
 
x_axis = Select(title="Time in seconds", options=sorted(axis_map.keys()), value="time")
y_axis = Select(title="WorkerID", options=sorted(axis_map.keys()), value="worker_id")
 
p = figure(plot_height=600, plot_width=700, title="", toolbar_location="below", tools=[hover,"pan","wheel_zoom","box_zoom"], toolbar_sticky=False)
p.hbar(y="y", height=1, left="acquire_lock_start", right="acquire_lock_end", source=source, color="#FF8633", legend="Acquire lock")
p.hbar(y="y", height=1, left="get_arguments_start", right="get_arguments_end", source=source, color="#8033FF", legend="Get arguments")
p.hbar(y="y", height=1, left="execute_start", right="execute_end", source=source, color="#3390FF", legend="Execute task")
p.hbar(y="y", height=1, left="store_outputs_start", right="store_outputs_end", source=source, color="#33FF9C", legend="Store outputs")
 
def select(lock_time, args_time, exec_time, outputs_time, worker_id_val, task_id_val, function_name_val):
    selected_tasks = dict()
    for task_id, data in task_profs.items():
        if ((data["acquire_lock_end"]-data["acquire_lock_start"]) > lock_time):
            if ((data["get_arguments_end"]-data["get_arguments_start"]) > args_time):
                if ((data["execute_end"]-data["execute_start"]) > exec_time):
                    if ((data["store_outputs_end"]-data["store_outputs_start"]) > outputs_time):
                        # these checks aren't actually right, whoops
                        if task_id_val:
                            if task_id_val in data["task_id"]:
                                selected_tasks[task_id] = data
                        elif worker_id_val:
                            if worker_id_val in data["worker_id"]:
                                selected_tasks[task_id] = data
                        elif function_name_val:
                            if function_name_val in data["function_name"]:
                                selected_tasks[task_id] = data 
                        else:
                            selected_tasks[task_id] = data
    return selected_tasks
   
def update(lock_time, args_time, exec_time, outputs_time, worker_id_val, task_id_val, function_name_val):
    selected_tasks = select(lock_time, args_time, exec_time, outputs_time, worker_id_val, task_id_val, function_name_val)
    print("update: selected {}".format(len(selected_tasks)))
    from collections import defaultdict
    df = defaultdict(list)
    for i, worker in selected_tasks.items():
        df["x"].append(worker["execute_start"])
        df["y"].append(worker["worker_id"])
        df["worker_id"].append(worker["worker_id"])
        df["task_id"].append(i)
        df["function_name"].append(worker["function_name"])
        df["get_arguments_start"].append(worker["get_arguments_start"])
        df["get_arguments_end"].append(worker["get_arguments_end"])
        df["acquire_lock_start"].append(worker["acquire_lock_start"])
        df['acquire_lock_end'].append(worker['acquire_lock_end'])
        df['store_outputs_start'].append(worker['store_outputs_start'])
        df['store_outputs_end'].append(worker['store_outputs_end'])
        df['execute_start'].append(worker['execute_start'])
        df['execute_end'].append(worker['execute_end'])
        df['wid'].append(worker["wid"])
        df['oid'].append(worker["object_id"])
 
    x_name = axis_map[x_axis.value]
    y_name = axis_map[y_axis.value]
    p.xaxis.axis_label = "Time in seconds"
    p.yaxis.axis_label = "Worker ID"
    p.title.text = "Task Information"
    source.data = dict(
        x= df["x"],
        y= df["y"],
        worker_id=df["worker_id"],
        task_id=df["task_id"],
        function_name=df["function_name"],
        acquire_lock_start=df["acquire_lock_start"],
        acquire_lock_end=df["acquire_lock_start"],
        get_arguments_start=df["get_arguments_start"],
        get_arguments_end=df["get_arguments_end"],
        store_outputs_start=df["store_outputs_start"],
        store_outputs_end=df["store_outputs_start"],
        execute_start=df["execute_start"],
        execute_end=df["execute_end"],
        wid=df["wid"],
        oid=df["oid"]
    )
    push_notebook()
    
show(p, notebook_handle=True)

from ipywidgets import interact
from ipywidgets import FloatSlider, Text

style = {"description_width": "initial"}
lock_time = FloatSlider(value=0, min=0, max=10, step=.1, description="Time to acquire lock: ", style=style)
args_time = FloatSlider(value=0, min=0, max=10, step=.1, description="Time to get arguments: ", style=style)
exec_time = FloatSlider(value=0, min=0, max=10, step=.1, description="Time to execute task: ", style=style)
outputs_time = FloatSlider(value=0, min=0, max=10, step=.1, description="Time to store outputs: ", style=style)
worker_id_val = Text(value="", description="Worker ID:",disabled=False)
task_id_val = Text(value="", description="Task ID:",disabled=False)
function_name_val = Text(value="", description="Function name:",disabled=False)

interact(update, lock_time=lock_time , args_time=args_time, exec_time=exec_time, outputs_time=outputs_time, 
         worker_id_val=worker_id_val, task_id_val=task_id_val, function_name_val=function_name_val)

### Task Distributions

In [ ]:
import numpy as np
import scipy.special

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.resources import CDN
from bokeh.io import output_notebook
output_notebook(resources=CDN)

hover = HoverTool(tooltips=[
    ("TaskID", "@task_id"),
    ("Function Name", "@function_name"),
    ("ParentID", "@parent_id"),
])

p1 = figure(title="Task Time Distribution (μ=0, σ=0.5)",tools=["save", "hover"],
            background_fill_color="#C0C0C0")

mu, sigma = 0, 0.5

task_profiles = ray.global_state.task_profiles()

task_times = []
for task_id, data in task_profiles.items(): 
    del data["worker_id"]
    del data["function_name"]
    time = max(data.values()) - min(data.values())
    task_times.append(time)

print("Task completion times:")
print(np.sort(task_times))
        
hist, edges = np.histogram(task_times, density=True, bins=50)

x = np.linspace(-2, 2, 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
p1.line(x, pdf, line_color="#AED19B", line_width=8, alpha=0.7, legend="PDF")
p1.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p1.legend.location = "top_right"
p1.legend.background_fill_color = "darkgrey"
p1.xaxis.axis_label = 'Time in ms'
p1.yaxis.axis_label = 'Pr(x)'

show(gridplot(p1, ncols=2, plot_width=600, plot_height=600, toolbar_location="below"))

### Task Dependency Graph

In [ ]:
import networkx as nx
from bokeh.plotting import figure, show
from bokeh.resources import CDN
from bokeh.io import output_notebook
output_notebook( resources=CDN )

edges = []
nodes = dict()

DG = nx.DiGraph()

task_info = ray.global_state.task_table()

source = ColumnDataSource(data=dict(
    x=[],
    y=[],
    task_id=[],
    parent_id=[], 
    function_name=[],
    errored=[]
#     object_id=[],
#     variables=[]
))

task_profiles = ray.global_state.task_profiles() 

for task_id, data in task_info.items(): 
    if task_id not in nodes:
        nodes[task_id] = True 
        parent_id = data["TaskSpec"]["ParentTaskID"]
        DG.add_node(task_id, task_id = task_id, parent_id = parent_id)
        edges.append((task_id, parent_id))
        
DG.add_edges_from(edges)
from collections import defaultdict
df = defaultdict(list)
pts = nx.spectral_layout(DG, scale=5)
# print(pts)
# print(task_profiles)
for task_id, indices in pts.items():
    if task_id in task_info:
        df["x"].append(indices[0])
        df["y"].append(indices[1])
        df["task_id"].append(task_info[task_id]["TaskSpec"]["TaskID"])
        if task_id in task_profiles: 
            df["function_name"].append(task_profiles[task_id]["function_name"])
        else: 
            df["function_name"].append("None")
        if "ParentTaskID" in task_info[task_id]["TaskSpec"]: 
            df["parent_id"].append(task_info[task_id]["TaskSpec"]["ParentTaskID"]) 
        else:
            df["parent_id"].append("None")
#         if task_id in error_profiles: 
#             df["errored"].append(True)
#         else: 
#             df["errored"].append(False) 

source.data = dict(
                x= df["x"],
                y= df["y"],
                parent_id=df["parent_id"],
                task_id=df["task_id"],
                function_name=df["function_name"], 
#                 errored=df["errored"]
                )

hover = HoverTool(tooltips=[
    ("TaskID", "@task_id"),
    ("Function Name", "@function_name"),
    ("ParentID", "@parent_id"),
#     ("ObjectID", "@oid"),
#     ("Variables", "@vars")
])

p = figure(
    x_range=(-5,5),
    y_range=(-5,5),
    height=700,
    width=700,
    tools=[hover,"pan","wheel_zoom","box_zoom"], 
    toolbar_sticky=False
)

p.xaxis.visible = False
p.yaxis.visible = False

p.line( 
    x="x",
    y="y",
    source=source
)

p.circle( 
x="x",
y="y",
source=source,
size=20)
    
show(p)

### Actor Information 


In [ ]:
actor_info = ray.global_state.actor_classes() 
for actor_id, data in actor_info.items():
    if "class" in data: 
        del data["class"]
actor_df = pd.DataFrame.from_dict(actor_info)
df = actor_df.T
df.index.name = "ActorID"
qgrid.show_grid(df)

# System State Information 


### Node Information

In [ ]:
ctable = ray.global_state.client_table()

client_list = []
for node_ip in ctable:
    for client in ctable[node_ip]:
        client["node_ip_address"] = node_ip
        client_list.append(client)

client_df = pd.DataFrame(client_list)
client_df.columns = ["Aux Address", "Client Type", "DB Client ID", "Deleted", "Local Scheduler Socket", "Num CPUs", "NumGPUs", "Node IP Address"]
qgrid.show_grid(client_df)

### Worker Information 

In [ ]:
worker_info = ray.global_state.workers() 
table = pd.DataFrame.from_dict(worker_info)
table_t = table.T
table_t.index.name = "WorkerID"
table_t.columns = ["Local Scheduler Socket", "Node IP Address", "Plasma Manager Socket", "Plasma Store Socket", "Stdout File"]
qgrid.show_grid(table_t)

### Object Store Information


In [ ]:
object_dict = {oid.hex(): v for oid, v in ray.global_state.object_table().items()}
if len(object_dict) == 0: 
    object_dict["None"] = {"Data Size":"None", "Hash":"None", "IsPut":"None", "ManagerIDs":"None", "TaskID":"None"}
object_df = pd.DataFrame(object_dict).transpose()
object_df.index.name = "ObjectID"
object_df.columns = ["Data Size", "Hash", "IsPut", "ManagerIDs", "TaskID"]
qgrid.show_grid(object_df)

# Error Information


In [ ]:
# TO-DO: This needs to be fixed so that it uses an API call rather than queries Redis
event_names = rc.keys("event_log*")
error_profiles = dict()
for i in range(len(event_names)):
    event_list = rc.lrange(event_names[i], 0, -1)
    for event in event_list:
        event_dict = json.loads(event)
        task_id = ""
        traceback = ""
        worker_id = ""
        start_time = -1
    for element in event_dict:
        if element[1] == "ray:task:execute" and element[2] == 1:
            start_time = element[0]
        if "task_id" in element[3] and "worker_id" in element[3]:
            task_id = element[3]["task_id"]
            worker_id = element[3]["worker_id"]
        if "traceback" in element[3]:
            traceback = element[3]["traceback"]
        if task_id != "" and worker_id != "" and traceback != "":
            if start_time != -1:
                error_profiles[task_id] = dict()
                error_profiles[task_id]["worker_id"] = worker_id
                error_profiles[task_id]["traceback"] = traceback
                error_profiles[task_id]["start_time"] = start_time
table = pd.DataFrame.from_dict(error_profiles) 
qgrid.show_grid(table.T)